In [22]:
!pip install lxml


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
!pip freeze > ..//requirements.txt

# regressores

In [50]:
import pandas as pd
import numpy as np
from datetime import datetime
df_conflitos = pd.read_csv('../data/raw/conflitos armados-2020-12-09-2023-12-26.csv')
df_conflitos['Date'] = [datetime.fromtimestamp(x).date() for x in df_conflitos.timestamp]
df_conflitos_porpais = df_conflitos\
    .groupby(['Date','event_type','country'])\
    .agg({
          'fatalities':'sum'
        , 'event_id_cnty':'nunique'
    }).reset_index()
df_conflitos_porpais.to_csv('../data/raw/df_conflitos_porpais.csv')
df_conflitos_mundiais = df_conflitos\
    .groupby(['Date','event_type'])\
    .agg({
          'fatalities':'sum'
        , 'event_id_cnty':'nunique'
    }).reset_index()
df_conflitos_mundiais.to_csv('../data/raw/df_conflitos_mundiais.csv')

In [2]:
df_conflitos[['event_type', 'sub_event_type']].drop_duplicates()

,event_type,sub_event_type
0,Battles,Armed clash
1,Violence against civilians,Attack
10,Explosions/Remote violence,Remote explosive/landmine/IED
11,Explosions/Remote violence,Air/drone strike
13,Explosions/Remote violence,Shelling/artillery/missile attack
280,Violence against civilians,Abduction/forced disappearance
306,Battles,Non-state actor overtakes territory
376,Violence against civilians,Sexual violence
453,Explosions/Remote violence,Grenade
464,Battles,Government regains territory


In [51]:
df_conflitos[['event_type', 'sub_event_type']].drop_duplicates().to_csv('../data/raw/tipo_subtipo_eventos.csv')

## conflitos e petroleo

In [53]:
df_conflitos_mundiais = pd.read_csv('../data/raw/df_conflitos_mundiais.csv')
df_conflitos_mundiais['Date'] = pd.to_datetime(df_conflitos_mundiais['Date'])
df_conflitos_mundiais = df_conflitos_mundiais\
.groupby(['Date','event_type'])\
.agg({
        'fatalities':'sum'
     , 'event_id_cnty':'sum'
}).reset_index()
df_conflitos_mundiais = df_conflitos_mundiais.pivot(index='Date', columns='event_type').reset_index().replace(0, None)
df_conflitos_mundiais.columns = ['Date','Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians']
df_conflitos_mundiais

,Date,Fatalities in Battles,Fatalities in Explosions/Remote Violence,Fatalities in Violence against civillians,Qtt Battles,Qtt Explosions/Remote violence,Qtt Violence against civilians
0,2020-12-14,7.0,1.0,7.0,3.0,10.0,11.0
1,2020-12-15,2.0,NaN,None,1.0,NaN,1.0
2,2021-01-06,8.0,NaN,18.0,13.0,NaN,15.0
3,2021-01-11,187.0,46.0,265.0,147.0,67.0,253.0
4,2021-01-12,7.0,3.0,16.0,9.0,10.0,27.0
...,...,...,...,...,...,...,...
312,2023-12-04,2656.0,9842.0,984.0,1183.0,1486.0,802.0
313,2023-12-05,637.0,345.0,130.0,420.0,905.0,158.0
314,2023-12-11,34621.0,10964.0,12354.0,10498.0,5306.0,8885.0
315,2023-12-12,1046.0,298.0,158.0,455.0,816.0,197.0


In [44]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', decimal=',', thousands='.', parse_dates=True)[2][1:]
df.columns=['Date','Preco']
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df = df.loc[df.Date.isin(df_conflitos_mundiais.Date.to_list())]
#------------
df_conflitos_mundiais = df_conflitos_mundiais[list(df_conflitos_mundiais['Date'] <= pd.to_datetime('2023-12-01'))]
df_conflitos_mundiais = df_conflitos_mundiais.merge(df, how='inner', on=['Date'])

for col in df_conflitos_mundiais.columns:
    if (col != 'Date'):
        df_conflitos_mundiais[col] = df_conflitos_mundiais[col].astype(float)
        df_conflitos_mundiais[f'log_{col}'] = np.log(df_conflitos_mundiais[col])


from sklearn.preprocessing import MinMaxScaler

cols_to_reescale = ['Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians', 'Preco']
scaler = MinMaxScaler()
df_conflitos_mundiais[["minmax_"+x for x in cols_to_reescale]] = scaler.fit_transform(df_conflitos_mundiais[cols_to_reescale])

In [45]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()

for col in cols_to_reescale:
    fig.add_trace(go.Scatter(
        x=df_conflitos_mundiais['Date'], y=df_conflitos_mundiais["minmax_"+col],
        mode='lines', yaxis='y', name=col.replace('minmax_','')
        #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
        )
    )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig

In [46]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()

for col in df_conflitos_mundiais.columns:
    if (col.startswith('log')):
        fig.add_trace(go.Scatter(
            x=df_conflitos_mundiais['Date'], y=df_conflitos_mundiais[col],
            mode='lines', yaxis='y', name=col.replace('log_','')
            #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
            )
        )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig

In [49]:
df_conflitos_mundiais['dt_year_month'] = [x.replace(day=1) for x in df_conflitos_mundiais.Date]
df_grouped_conf_mund = df_conflitos_mundiais.groupby('dt_year_month').agg({
    'Fatalities in Battles':'sum'
    ,'Fatalities in Explosions/Remote Violence':'sum'
    ,'Fatalities in Violence against civillians':'sum'
    ,'Qtt Battles':'sum'
    ,'Qtt Explosions/Remote violence':'sum'
    ,'Qtt Violence against civilians':'sum'
    ,'Preco':'mean'
}).reset_index()
df_grouped_conf_mund['Total Fatalities'] = df_grouped_conf_mund['Fatalities in Battles'] + df_grouped_conf_mund['Fatalities in Explosions/Remote Violence'] + df_grouped_conf_mund['Fatalities in Violence against civillians']
df_grouped_conf_mund['Total Qtt'] = df_grouped_conf_mund['Qtt Battles'] + df_grouped_conf_mund['Qtt Explosions/Remote violence'] + df_grouped_conf_mund['Qtt Violence against civilians']
cols_to_reescale = ['Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians', 'Preco','Total Qtt','Total Fatalities']
scaler = MinMaxScaler()
df_grouped_conf_mund[["minmax_"+x for x in cols_to_reescale]] = scaler.fit_transform(df_grouped_conf_mund[cols_to_reescale])

fig = go.Figure()

for col in cols_to_reescale:
    fig.add_trace(go.Scatter(
        x=df_grouped_conf_mund['dt_year_month'], y=df_grouped_conf_mund["minmax_"+col],
        mode='lines', yaxis='y', name=col.replace('minmax_','')
        #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
        )
    )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig

# Uso de energia

In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler

cols = [str(x) for x in ['Country Name'] + list(range(1960, 2023))]


In [103]:
df_uso_energia = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/energy_use/API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_6301176.csv')

df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', decimal=',', thousands='.', parse_dates=True)[2][1:]
df.columns=['Date','Preco']
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df_petroleo = df.sort_values('Date')

# ajustar df de petroleo
df_petroleo['year'] = [str(x.year) for x in df_petroleo['Date']]
df_petroleo['Preco'] = df_petroleo['Preco'].astype(float) 
df_petroleo_year = df_petroleo.groupby('year').agg({'Preco':'mean'}).reset_index()
df_petroleo_year['Country Name'] = 'Price'

# colunas em comum
cols = list(set(df_uso_energia.columns).intersection(df_petroleo_year.year.values) - set(['Country Name']))
df_petroleo_year = df_petroleo_year.loc[df_petroleo_year.year.isin(cols)]

# normalizar apenas o preço que estará no comparativo
scaler_py = MinMaxScaler()
df_petroleo_year[["minmax_Preco"]] = scaler_py.fit_transform(df_petroleo_year[['Preco']])
# # pivotar para unir com df_uso_energia
df_petroleo_year = df_petroleo_year.pivot(index='Country Name', columns='year',values='minmax_Preco').reset_index()
df_petroleo_year.columns = [str(x) for x in df_petroleo_year.columns]

# # colunas em comum
# cols = list(set(df_uso_energia.columns).intersection(df_petroleo_year.columns))
df_uso_energia = df_uso_energia[['Country Name'] + cols]#pd.concat([df_uso_energia[cols],df_petroleo_year[cols]])

# vamos preencher o na de 2015 de alguns paises com a media dos anos de 11-15
df_uso_energia['mean'] = [round(x, 2) for x in df_uso_energia[['2011','2012','2013','2014','2015']].mean(axis=1)]
df_uso_energia['2015_nulo'] = np.where(np.isnan(df_uso_energia['2015']), 1, 0)
df_uso_energia['2015']  = np.where(np.isnan(df_uso_energia['2015']), df_uso_energia['mean'], df_uso_energia['2015'])
lista_2015_nulo = [f'-{x}' for x in df_uso_energia.loc[df_uso_energia['2015_nulo']==1]['Country Name'].values]

# reescalar pra comparação
cols_to_reescale = list(set(df_uso_energia.columns) - set(['Country Name']))
scaler_en = MinMaxScaler()
df_uso_energia[["minmax_"+x for x in cols_to_reescale]] = scaler_en.fit_transform(df_uso_energia[cols_to_reescale])


# pegar apenas colunas de interesse
cols = ['Country Name'] + [x for x in df_uso_energia.columns if x.startswith('minmax_')]
df_uso_energia = df_uso_energia[cols]
cols = [x.replace('minmax_','') for x in cols]
df_uso_energia.columns = cols
df_uso_energia = pd.concat([df_uso_energia[cols],df_petroleo_year[cols]])

# # # reshape pra plotar
cols_to_plot = [x for x in list(set(df_uso_energia.columns)-set(['Country Name']))]
df_uso_energia = df_uso_energia[['Country Name'] + cols_to_plot]
df_uso_energia = df_uso_energia.melt(id_vars='Country Name').rename(columns={'variable':'Year'}).sort_values('Year')

d:\Cursos\FIAP_pós\gp27_techchallenge_4\.venv\lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning:

All-NaN slice encountered

d:\Cursos\FIAP_pós\gp27_techchallenge_4\.venv\lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning:

All-NaN slice encountered



In [106]:
fig = go.Figure()


# fig.add_trace(go.Scatter(
#     x=df_uso_energia['Year'], y=df_uso_energia['Country Name'],
#     mode='lines', yaxis='y'#, name=col
#     #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
#     )
# )
fig = px.line(
    df_uso_energia, 
    x='Year', y='value', color='Country Name'
    #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
)

# hide axes
fig.update_layout(
    hovermode='x unified',
)

In [153]:
df_uso_energia_or = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/energy_use/API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_6301176.csv').drop(columns={'Indicator Name','Indicator Code'})
df_country_region = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/energy_use/Metadata_Country_API_EG.USE.PCAP.KG.OE_DS2_en_csv_v2_6301176.csv')[['Country Code','Region']].dropna()
df_uso_energia_or = df_uso_energia_or.merge(df_country_region, how='inner', on='Country Code').drop(columns={'Country Code','Region'})
cols_to_plot.sort(reverse=True)
df_uso_energia_or = df_uso_energia_or.dropna(axis=1, thresh=0.95)

cols_to_plot = [x for x in list(set(df_uso_energia_or.columns)-set(['Country Name']))]
cols_to_plot.sort(reverse=True)
cols_to_plot = cols_to_plot[:5]
df_uso_energia_or = df_uso_energia_or[['Country Name'] + cols_to_plot]
df_uso_energia_or['mean'] = [round(x, 2) for x in df_uso_energia_or[cols_to_plot].mean(axis=1)]
df_uso_energia_or['Total En. Use 11-15'] = [round(x, 2) for x in df_uso_energia_or[cols_to_plot].sum(axis=1)]
df_uso_energia_or = df_uso_energia_or\
    .sort_values('Total En. Use 11-15', ascending=False)\
    .head(10)
df_uso_energia_or['2015'] = np.where(np.isnan(df_uso_energia_or['2015']), df_uso_energia_or['mean'], df_uso_energia_or['2015'])
df_uso_energia_or

,Country Name,2015,2014,2013,2012,2011,mean,Total En. Use 11-15
93,Iceland,17478.893037,17916.120421,18178.139015,17630.071465,18157.598099,17872.16,89360.82
163,Qatar,19263.910000,19903.475106,19819.982894,19987.584354,17344.592613,19263.91,77055.63
195,Trinidad and Tobago,13613.300000,13489.037756,13604.316981,13456.276912,13903.561217,13613.30,54453.19
46,Curacao,13459.450000,12650.860438,11727.509719,13395.369786,16064.051820,13459.45,53837.79
19,Bahrain,10590.690000,10798.122846,10860.468600,10251.069645,10453.083426,10590.69,42362.74
106,Kuwait,9960.680000,9006.504972,9580.126576,10956.105805,10299.968987,9960.68,39842.71
32,Canada,7631.341570,7897.855615,7743.725742,7733.411655,7911.554588,7783.58,38917.89
116,Luxembourg,6548.405559,6861.106667,7312.154005,7722.190204,8056.404301,7300.05,36500.26
28,Brunei Darussalam,8745.940000,8529.729561,7389.223273,9418.902502,9645.895703,8745.94,34983.75
203,United States,6803.996607,6960.683997,6905.598633,6872.027284,7029.954601,6914.45,34572.26


# fuel consumption

In [160]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', decimal=',', thousands='.', parse_dates=True)[2][1:]
df.columns=['Date','Preco']
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df_petroleo = df.sort_values('Date')

# ajustar df de petroleo
df_petroleo['Year'] = [str(x.year) for x in df_petroleo['Date']]
df_petroleo['Preco'] = df_petroleo['Preco'].astype(float) 
df_petroleo_year = df_petroleo.groupby('Year').agg({'Preco':'mean'}).reset_index()
df_petroleo_year['Country Name'] = 'Price'

In [189]:
#  selecionar colunas numericas e nome
cols = [str(x) for x in ['Country Name','Region'] + list(range(1960, 2023))]
df_fuel_cons = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fossil_fuel_consumption/API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_6299038.csv')
df_country_region = pd.read_csv('D:/Cursos/FIAP_pós/gp27_techchallenge_4/data/raw/fossil_fuel_consumption/Metadata_Country_API_EG.USE.COMM.FO.ZS_DS2_en_csv_v2_6299038.csv')[['Country Code','Region']].dropna()
df_fuel_cons = df_fuel_cons.merge(df_country_region, how='inner', on='Country Code').drop(columns={'Country Code'})[cols]

# dropar na
df_fuel_cons = df_fuel_cons.dropna(axis=1, thresh=0.95)
cols_to_plot = [x for x in list(set(df_fuel_cons.columns)-set(['Country Name','Region']))]
cols_to_plot.sort(reverse=True)
cols_to_plot = cols_to_plot[:5]
df_fuel_cons = df_fuel_cons[['Country Name','Region'] + cols_to_plot[:5]]

# top10
df_fuel_cons['Mean Fuel Cons. 11-15'] = [round(x, 2) for x in df_fuel_cons[cols_to_plot].mean(axis=1)]
df_fuel_cons['Total Fuel Cons. 11-15'] = [round(x, 2) for x in df_fuel_cons[cols_to_plot].sum(axis=1)]
df_fuel_cons = df_fuel_cons\
    .sort_values('Mean Fuel Cons. 11-15', ascending=False)\
    .head(10)


In [190]:
df_fuel_cons

,Country Name,Region,2015,2014,2013,2012,2011,Mean Fuel Cons. 11-15,Total Fuel Cons. 11-15
28,Brunei Darussalam,East Asia & Pacific,NaN,99.995188,99.994379,99.995535,99.995636,100.00,399.98
163,Qatar,Middle East & North Africa,NaN,99.995027,100.000000,NaN,NaN,100.00,200.00
149,Oman,Middle East & North Africa,NaN,99.963169,100.000000,NaN,100.000000,99.99,299.96
195,Trinidad and Tobago,Latin America & Caribbean,NaN,99.919506,99.929949,99.928048,99.929242,99.93,399.71
55,Algeria,Middle East & North Africa,NaN,99.977917,99.910365,99.858917,99.887476,99.91,399.63
167,Saudi Arabia,Middle East & North Africa,NaN,99.928019,99.570562,99.510852,99.203692,99.55,398.21
71,Gibraltar,Europe & Central Asia,NaN,99.512817,98.960509,NaN,100.000000,99.49,298.47
91,"Iran, Islamic Rep.",Middle East & North Africa,NaN,99.022581,98.958888,99.324685,99.417129,99.18,396.72
19,Bahrain,Middle East & North Africa,NaN,99.365847,NaN,NaN,98.785946,99.08,198.15
110,Libya,Middle East & North Africa,NaN,99.111296,99.185506,99.081353,98.902751,99.07,396.28


In [163]:

# colunas em comum
cols = list(set(df_fuel_cons.columns).intersection(df_petroleo_year.Year.values) - set(['Country Name']))
df_petroleo_year = df_petroleo_year.loc[df_petroleo_year.Year.isin(cols)]

# normalizar apenas o preço que estará no comparativo
scaler_py = MinMaxScaler()
df_petroleo_year[["value"]] = scaler_py.fit_transform(df_petroleo_year[['Preco']])
# pivotar para unir com df_uso_energia
df_petroleo_year = df_petroleo_year[['Country Name','Year','value']]
df_petroleo_year.columns = [str(x) for x in df_petroleo_year.columns]